In [103]:
# Parameter Cell
object_name="VAS"
instance_name="czpoh_2"

config_path = 'abfss://development@dscglblceedadlstpldnsazr.dfs.core.windows.net/config'
bronze_path = f'abfss://production@dscglblceedadlsppldnsazr.dfs.core.windows.net/bronze/blueyonder_wms/{instance_name}'
silver_path = f'abfss://development@dscglblceedadlstpldnsazr.dfs.core.windows.net/silver/{instance_name}'
gold_path = f'abfss://development@dscglblceedadlstpldnsazr.dfs.core.windows.net/gold/{instance_name}'

StatementMeta(dscglceedaspt, 1894, 104, Finished, Available)

In [104]:
columns = ['LODNUM',
           'TRNDTE',
           'ACTCOD',
           'SUBNUM',
           'PRTNUM',
           'TRNQTY',
           'FR_ARECOD',
           'TO_ARECOD',
           'TOSTOL',
           'USR_ID',
           'EXPQTY',
           'RPTQTY']

           
ACTCOD = ['CS_XRF',
          'GENMOV'
          'PL_XFR'
          'FL_XFR'
          'ASN_REMOVE']

StatementMeta(dscglceedaspt, 1894, 105, Finished, Available)

## Imports


In [105]:
import numpy as np
import delta.tables as DT
import pyspark.sql.functions as F
import pyspark.sql.types as T
from datetime import datetime

StatementMeta(dscglceedaspt, 1894, 106, Finished, Available)

## Loading Data

In [106]:
df_DLYTRN= (spark
.read
.format("delta")
.load(f"{bronze_path}/DLYTRN")
.select(columns)
.where("ACTCOD = 'CS_XRF' OR ACTCOD = 'GENMOV' OR ACTCOD = 'PL_XFR' OR ACTCOD = 'FL_XFR' OR ACTCOD = 'ASN_REMOVE' "))

df_CALMST= (
spark
.read
.format("delta")
.load(f"{bronze_path}/CALMST").select(F.col('DCSDTE'), F.col('WH_ID'), F.col('WRKFLG'))
.where("DW_VALID_FLAG = TRUE"))


StatementMeta(dscglceedaspt, 1894, 107, Finished, Available)

## PHOTOSHOP_IN
#### Geting start date, number of items and other needed columns
 



In [107]:
df_DLYTRN_IN = (df_DLYTRN
.filter((F.col('TOSTOL') == 'PHOTOSHOP-IN'))
.groupBy('LODNUM')
.agg(
     F.min(F.col("TRNDTE")).alias('DW_START_DATE'),
     F.sum('TRNQTY').astype(T.LongType()).alias('DW_MOVED_QNTY'),
     F.count("PRTNUM").astype(T.LongType()).alias('DW_NBR_ITEMS'),
     F.collect_list('SUBNUM').alias("DW_SUBNUM_SET")))
     #3659

StatementMeta(dscglceedaspt, 1894, 108, Finished, Available)

## PHOTOSHOP-OUT
#### Geting end date and reference date


In [108]:
df_DLYTRN_OUT = (df_DLYTRN
.filter((F.col('TOSTOL') == 'PHOTOSHOP-OUT-OPEN')|(F.col('TOSTOL') == 'PHOTOSHOP-OUT-LOT'))
.groupBy('LODNUM')
.agg( 
    F.min(F.col("TRNDTE")).alias('DW_END_DATE'),
    F.min(F.col("TRNDTE")).alias('REFERRENCE_DTE')))
    #12479

StatementMeta(dscglceedaspt, 1894, 109, Finished, Available)

## Joins and calculations
#### adding the original dataset to get additional columns

In [109]:
df_DLYTRN_All = (df_DLYTRN_OUT
.join(df_DLYTRN_IN,['LODNUM'],'inner')
.withColumn('REFERRENCE_DTE',F.to_date(F.col("REFERRENCE_DTE"),"MM-dd-yyyy"))
.withColumn('DW_DURATION', (F.col('DW_END_DATE').cast("long")-F.col('DW_START_DATE').cast("long"))/3600))

StatementMeta(dscglceedaspt, 1894, 110, Finished, Available)

#####  CALMST EXCLUDED DURATION

In [110]:
#Creating a joinable CALMST
CALMST_list = list(df_CALMST.select(F.date_format(F.col('DCSDTE'), "yyyy-MM-dd").alias("DCSDTE")).toPandas() ['DCSDTE'])
CALMST_list = str(CALMST_list)[1:-1] #list of dates without square brackets

joinable_CALMST = (df_DLYTRN_All
.select(
    F.col('DW_START_DATE').cast(T.DateType()),
    F.col('DW_END_DATE').cast(T.DateType()),
    F.col('LODNUM'),
F.explode(F.when(F.col('DW_START_DATE')<F.col('DW_END_DATE'),F.expr('sequence(DW_START_DATE, DW_END_DATE, interval 1 day)'))))\
.groupBy('DW_START_DATE', 'DW_END_DATE','LODNUM')
     .agg(F.collect_set(F.date_format(F.col('col'), "yyyy-MM-dd")).alias("DW_list_dates"))\
.withColumn('DW_EXCLUDE_DURATION', F.when(F.col('DW_START_DATE')>=F.col('DW_END_DATE'),0)\
.otherwise(F.size(F.array_intersect(F.col('DW_list_dates'),F.expr(f"array({CALMST_list})")))*24))
.select('LODNUM','DW_EXCLUDE_DURATION'))


StatementMeta(dscglceedaspt, 1894, 111, Finished, Available)

In [112]:
df_DLYTRN_All = (df_DLYTRN_All
.join(joinable_CALMST,['LODNUM'],'left')
.withColumn('DW_ADJ_DURATION', (F.col('DW_DURATION')-F.col('DW_EXCLUDE_DURATION')))
.select( 'LODNUM',
         'DW_SUBNUM_SET',
         'DW_START_DATE',
         'DW_END_DATE',
         'DW_DURATION',
         'DW_EXCLUDE_DURATION',
         'DW_ADJ_DURATION',
         'DW_NBR_ITEMS',
         'DW_MOVED_QNTY',
         'REFERRENCE_DTE'))

StatementMeta(dscglceedaspt, 1894, 113, Finished, Available)

In [96]:
#display(df_DLYTRN_All)

StatementMeta(dscglceedaspt, 1894, 97, Finished, Available)

SynapseWidget(Synapse.DataFrame, 63fa8e26-27ff-486b-bc40-3a370058832f)

## Duplication check


In [99]:
#df_DLYTRN_All.select('LODNUM').distinct().count()#8870

StatementMeta(dscglceedaspt, 1894, 100, Finished, Available)

In [100]:
#df_DLYTRN_All.select('LODNUM').count()#8870

StatementMeta(dscglceedaspt, 1894, 101, Finished, Available)

In [101]:
#print(df_DLYTRN_All.count()== df_DLYTRN_All.distinct().count())#True

StatementMeta(dscglceedaspt, 1894, 102, Finished, Available)

## Saving to Silver


In [113]:
df_DLYTRN_All\
.write\
.format("delta")\
.mode("overwrite")\
.option("overwriteSchema", "true")\
.save(f"{silver_path}/{object_name}")

StatementMeta(dscglceedaspt, 1894, 114, Finished, Available)